# PhysicsNeMo-Curator Ansys Tutorial

## Overview

This section contains a tutorial for using PhysicsNeMo-Curator to prepare a training ready dataset from Ansys Solvers using the PyAnsys libraries.
This tutorial will show how to use the PhysicsNeMo-Curator ETL pipeline to:

1. Extract data from Ansys `.rst` files
2. Transform the data into an optimized, AI model training ready format
3. Write the transformed data to disk efficiently

# Step 1: Installation of prerequisites

In [ ]:
pip install -e "../../[dev]" ansys-dpf-core --user

Obtaining file:///workspace/physics_nemo/physicsnemo-curator-saikrishnanc-fork
  DEPRECATION: Setting PIP_CONSTRAINT will not affect build constraints in the future, pip 26.2 will enforce this behaviour change. A possible replacement is to specify build constraints using --build-constraint or PIP_BUILD_CONSTRAINT. To disable this warning without any build constraints set --use-feature=build-constraint or PIP_USE_FEATURE="build-constraint".
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for physicsnemo_curator (pyproject.toml) ... done
  Created wheel for physicsnemo_curator: filename=physicsnemo_curator-0.0.1-0.editable-py3-none-any.whl size=10025 sha256=153723a6c17117a0bd793619b397cc3a0df3ef930b837a7019fd20e3773cb970
  Stored in directory: /tmp/pip-ephem-wheel-cache-wxwlzu2r/wheels/08/d5/29/eff3d737047abd57

# Step 2: Generate sample data

We will generate mock .rst data, since using real data requires a license (instructions for adapting to real data are provided at the end of this tutorial).

You can generate mock data by running the script below.

In [ ]:
import pickle  # noqa: F811
from pathlib import Path
from typing import Any, Dict

import numpy as np


def generate_3d_mesh(nx: int, ny: int, nz: int, 
                     domain_size: tuple = (1.0, 1.0, 1.0)) -> np.ndarray:
    """
    Generate a regular 3D mesh grid.
    
    Args:
        nx, ny, nz: Number of nodes in x, y, z directions
        domain_size: Physical size of the domain in meters (x, y, z)
    
    Returns:
        coordinates: (N, 3) array of node coordinates
    """
    x = np.linspace(0, domain_size[0], nx)
    y = np.linspace(0, domain_size[1], ny)
    z = np.linspace(0, domain_size[2], nz)
    
    X, Y, Z = np.meshgrid(x, y, z, indexing='ij')
    coordinates = np.stack([X.ravel(), Y.ravel(), Z.ravel()], axis=1)
    
    return coordinates


def generate_temperature_field(coordinates: np.ndarray, 
                                scenario: str = "heat_source") -> np.ndarray:
    """
    Generate a temperature field based on different thermal scenarios.
    
    Args:
        coordinates: (N, 3) array of node coordinates
        scenario: Type of thermal scenario
    
    Returns:
        temperature: (N,) array of temperature values in Kelvin
    """
    x, y, z = coordinates[:, 0], coordinates[:, 1], coordinates[:, 2]
    
    if scenario == "heat_source":
        # Heat source at the center
        center = np.array([0.5, 0.5, 0.5])
        distance = np.sqrt((x - center[0])**2 + (y - center[1])**2 + (z - center[2])**2)
        temperature = 300 + 100 * np.exp(-10 * distance**2)
        
    elif scenario == "linear_gradient":
        # Linear temperature gradient in x-direction
        temperature = 300 + 50 * x
        
    elif scenario == "hot_cold_sides":
        # Hot on left, cold on right
        temperature = 350 - 100 * x
        
    elif scenario == "corner_heating":
        # Multiple heat sources at corners
        corners = [
            np.array([0.1, 0.1, 0.1]),
            np.array([0.9, 0.9, 0.9])
        ]
        temperature = np.ones(len(coordinates)) * 300
        for corner in corners:
            distance = np.sqrt((x - corner[0])**2 + (y - corner[1])**2 + (z - corner[2])**2)
            temperature += 80 * np.exp(-20 * distance**2)
    
    elif scenario == "periodic_heating":
        # Periodic heating pattern
        temperature = 300 + 50 * np.sin(4 * np.pi * x) * np.cos(4 * np.pi * y)
    
    else:
        # Default uniform temperature
        temperature = np.ones(len(coordinates)) * 300
    
    # Add some random noise to make it more realistic
    temperature += np.random.normal(0, 2, len(coordinates))
    
    return temperature


def generate_heat_flux(coordinates: np.ndarray, 
                       temperature: np.ndarray,
                       thermal_conductivity: float = 50.0) -> np.ndarray:
    """
    Generate heat flux vectors based on temperature gradients.
    
    Uses a simplified approach: flux ∝ -∇T (Fourier's law)
    
    Args:
        coordinates: (N, 3) array of node coordinates
        temperature: (N,) array of temperatures
        thermal_conductivity: Material thermal conductivity (W/m·K)
    
    Returns:
        heat_flux: (N, 3) array of heat flux vectors (W/m²)
    """
    # Compute approximate temperature gradients using finite differences
    # This is a simplified approach for mock data
    
    n_nodes = len(coordinates)
    heat_flux = np.zeros((n_nodes, 3))
    
    # Simple gradient approximation based on position
    x, y, z = coordinates[:, 0], coordinates[:, 1], coordinates[:, 2]
    
    # Approximate gradients (this is mock data, so we use a simple approach)
    # In reality, this would require proper finite element calculations
    dT_dx = np.gradient(temperature.reshape(-1), x.reshape(-1))[0] if len(x) > 1 else 0
    dT_dy = np.gradient(temperature.reshape(-1), y.reshape(-1))[0] if len(y) > 1 else 0
    dT_dz = np.gradient(temperature.reshape(-1), z.reshape(-1))[0] if len(z) > 1 else 0
    
    # Fourier's law: q = -k * ∇T
    heat_flux[:, 0] = -thermal_conductivity * dT_dx
    heat_flux[:, 1] = -thermal_conductivity * dT_dy
    heat_flux[:, 2] = -thermal_conductivity * dT_dz
    
    # Add some randomness to make it more realistic
    heat_flux += np.random.normal(0, 50, heat_flux.shape)
    
    return heat_flux


def generate_thermal_simulation(simulation_name: str,
                                mesh_resolution: tuple = (10, 10, 10),
                                scenario: str = "heat_source") -> Dict[str, Any]:
    """
    Generate a complete thermal simulation dataset.
    
    Args:
        simulation_name: Name identifier for this simulation
        mesh_resolution: Number of nodes in (x, y, z)
        scenario: Thermal scenario type
    
    Returns:
        data: Dictionary containing all simulation data
    """
    # Generate mesh
    coordinates = generate_3d_mesh(*mesh_resolution)
    n_nodes = len(coordinates)
    n_elements = (mesh_resolution[0] - 1) * (mesh_resolution[1] - 1) * (mesh_resolution[2] - 1)
    
    # Generate temperature field
    temperature = generate_temperature_field(coordinates, scenario)
    
    # Generate heat flux
    heat_flux = generate_heat_flux(coordinates, temperature)
    
    # Create metadata
    metadata = {
        "simulation_name": simulation_name,
        "num_nodes": n_nodes,
        "num_elements": max(n_elements, 1),  # Ensure at least 1
        "mesh_resolution": mesh_resolution,
        "scenario": scenario,
        "units": "SI",
        "temperature_units": "Kelvin",
        "heat_flux_units": "W/m^2",
        "coordinate_units": "meters",
        "time_step": 1,
        "analysis_type": "Steady-State Thermal",
        "solver": "Mock Thermal Solver v1.0",
        "temperature_min": float(temperature.min()),
        "temperature_max": float(temperature.max()),
        "temperature_mean": float(temperature.mean()),
    }
    
    data = {
        "coordinates": coordinates,
        "temperature": temperature,
        "heat_flux": heat_flux,
        "metadata": metadata,
    }
    
    return data


def save_simulation(data: Dict[str, Any], output_path: Path) -> None:
    """
    Save simulation data to disk.
    
    Args:
        data: Simulation data dictionary
        output_path: Path to save the data
    """
    with open(output_path, 'wb') as f:
        pickle.dump(data, f)
    print(f"Saved simulation to: {output_path}")


def main():
    """Generate 5 different thermal simulation datasets."""
    
    # Create output directory
    output_dir = Path("mock_thermal_data")
    output_dir.mkdir(exist_ok=True)
    
    print("=" * 60)
    print("Generating Mock Thermal Simulation Data")
    print("=" * 60)
    
    # Define 5 different simulation scenarios
    simulations = [
        {
            "name": "thermal_sim_001",
            "resolution": (12, 12, 12),
            "scenario": "heat_source",
            "description": "Central heat source with radial heat dissipation"
        },
        {
            "name": "thermal_sim_002",
            "resolution": (15, 10, 10),
            "scenario": "linear_gradient",
            "description": "Linear temperature gradient"
        },
        {
            "name": "thermal_sim_003",
            "resolution": (10, 10, 15),
            "scenario": "hot_cold_sides",
            "description": "Hot and cold boundary conditions"
        },
        {
            "name": "thermal_sim_004",
            "resolution": (14, 14, 8),
            "scenario": "corner_heating",
            "description": "Multiple localized heat sources"
        },
        {
            "name": "thermal_sim_005",
            "resolution": (16, 16, 10),
            "scenario": "periodic_heating",
            "description": "Periodic heating pattern"
        },
    ]
    
    # Generate each simulation
    for sim_config in simulations:
        print(f"\nGenerating: {sim_config['name']}")
        print(f"  Description: {sim_config['description']}")
        print(f"  Resolution: {sim_config['resolution']}")
        
        data = generate_thermal_simulation(
            simulation_name=sim_config['name'],
            mesh_resolution=sim_config['resolution'],
            scenario=sim_config['scenario']
        )
        
        # Save to disk with .rst extension (mimicking real Ansys format)
        output_path = output_dir / f"{sim_config['name']}.rst"
        save_simulation(data, output_path)
        
        print(f"  Nodes: {data['metadata']['num_nodes']}")
        print(f"  Temperature range: {data['metadata']['temperature_min']:.1f} - "
              f"{data['metadata']['temperature_max']:.1f} K")
    
    print("\n" + "=" * 60)
    print("Data generation complete!")
    print(f"Generated {len(simulations)} thermal simulation files (.rst) in: {output_dir}")
    print("=" * 60)
    print("\nThese mock .rst files can be processed by the ETL pipeline.")
    print("To use real Ansys .rst files, you would need:")
    print("  - Ansys installation (2021 R1+)")
    print("  - Valid Ansys license")
    print("  - PyDPF-Core configured with DPF Server")


if __name__ == "__main__":
    main()



Generating Mock Thermal Simulation Data

Generating: thermal_sim_001
  Description: Central heat source with radial heat dissipation
  Resolution: (12, 12, 12)
Saved simulation to: mock_thermal_data/thermal_sim_001.rst
  Nodes: 1728
  Temperature range: 294.9 - 395.5 K

Generating: thermal_sim_002
  Description: Linear temperature gradient
  Resolution: (15, 10, 10)
Saved simulation to: mock_thermal_data/thermal_sim_002.rst
  Nodes: 1500
  Temperature range: 295.9 - 355.5 K

Generating: thermal_sim_003
  Description: Hot and cold boundary conditions
  Resolution: (10, 10, 15)
Saved simulation to: mock_thermal_data/thermal_sim_003.rst
  Nodes: 1500
  Temperature range: 244.5 - 354.6 K

Generating: thermal_sim_004
  Description: Multiple localized heat sources
  Resolution: (14, 14, 8)
Saved simulation to: mock_thermal_data/thermal_sim_004.rst
  Nodes: 1568
  Temperature range: 293.5 - 374.6 K

Generating: thermal_sim_005
  Description: Periodic heating pattern
  Resolution: (16, 16, 10)

/usr/local/lib/python3.12/dist-packages/numpy/lib/function_base.py:1242: RuntimeWarning: divide by zero encountered in divide
  a = -(dx2)/(dx1 * (dx1 + dx2))
/usr/local/lib/python3.12/dist-packages/numpy/lib/function_base.py:1242: RuntimeWarning: invalid value encountered in divide
  a = -(dx2)/(dx1 * (dx1 + dx2))
/usr/local/lib/python3.12/dist-packages/numpy/lib/function_base.py:1243: RuntimeWarning: divide by zero encountered in divide
  b = (dx2 - dx1) / (dx1 * dx2)
/usr/local/lib/python3.12/dist-packages/numpy/lib/function_base.py:1243: RuntimeWarning: invalid value encountered in divide
  b = (dx2 - dx1) / (dx1 * dx2)
/usr/local/lib/python3.12/dist-packages/numpy/lib/function_base.py:1244: RuntimeWarning: divide by zero encountered in divide
  c = dx1 / (dx2 * (dx1 + dx2))
/usr/local/lib/python3.12/dist-packages/numpy/lib/function_base.py:1244: RuntimeWarning: invalid value encountered in divide
  c = dx1 / (dx2 * (dx1 + dx2))
/usr/local/lib/python3.12/dist-packages/numpy/lib/fun


# Step 3: Implement a datasource

We will implement a datasource to read the .rst files we just produced.
Instructions are provided in the docstring to adapt this to real ANSYS .rst files.


In [ ]:
import pickle  # noqa: F811
from pathlib import Path
from typing import Any, Dict, List

from physicsnemo_curator.etl.data_sources import DataSource
from physicsnemo_curator.etl.processing_config import ProcessingConfig


class RstDataSource(DataSource):
    """
    DataSource for reading thermal simulation .rst files.
    
    This implementation reads mock .rst files (pickled Python data) for tutorial purposes.
    
    ============================================================================
    USING REAL ANSYS .RST FILES:
    ============================================================================
    To adapt this for real Ansys .rst files, you need:
    
    1. Install Ansys (2021 R1 or later)
    2. Have a valid Ansys license
    3. Install PyDPF-Core: pip install ansys-dpf-core
    4. Replace the read_file() method with PyDPF code:
    
        from ansys.dpf import core as dpf
        
        def read_file(self, filename: str) -> Dict[str, Any]:
            filepath = self.input_dir / f"{filename}.rst"
            model = dpf.Model(str(filepath))
            
            # Extract mesh and temperature data
            mesh = model.metadata.meshed_region
            coords = np.array(mesh.nodes.coordinates_field.data)
            
            # Extract temperature (for thermal analysis)
            temp_op = model.results.temperature()
            temp_fields = temp_op.outputs.fields_container()
            temperature = np.array(temp_fields[0].data)
            
            # Extract heat flux if available
            flux_op = model.results.heat_flux()
            flux_fields = flux_op.outputs.fields_container()
            heat_flux = np.array(flux_fields[0].data)
            
            return {
                "coordinates": coords,
                "temperature": temperature,
                "heat_flux": heat_flux,
                "metadata": {...},
                "filename": filename
            }
    
    For more info: https://dpf.docs.pyansys.com/
    ============================================================================
    """

    def __init__(self, cfg: ProcessingConfig, input_dir: str):
        super().__init__(cfg)
        self.input_dir = Path(input_dir)

        if not self.input_dir.exists():
            raise FileNotFoundError(f"Input directory {self.input_dir} does not exist") 

    def get_file_list(self) -> List[str]:
        """Find all .rst files in the input directory."""
        rst_files = list(self.input_dir.glob("*.rst"))
        filenames = [f.stem for f in rst_files]
        self.logger.info(f"Found {len(filenames)} .rst files to process")
        return sorted(filenames)

    def read_file(self, filename: str) -> Dict[str, Any]:
        """
        Read a mock thermal .rst file.
        
        For this tutorial, .rst files are pickled Python dictionaries containing:
        - coordinates: (N, 3) array of node positions
        - temperature: (N,) array of temperature values in Kelvin
        - heat_flux: (N, 3) array of heat flux vectors in W/m²
        - metadata: dict with simulation information
        
        Returns data in a format ready for transformation.
        """
        filepath = self.input_dir / f"{filename}.rst"
        self.logger.info(f"Reading thermal simulation from: {filepath}")

        # Load the pickled thermal data
        with open(filepath, 'rb') as f:
            data = pickle.load(f)  # noqa: S301
        
        # Add filename for tracking
        data["filename"] = filename
        
        # Log basic info
        self.logger.info(f"  Loaded {data['metadata']['num_nodes']} nodes")
        self.logger.info(f"  Temperature range: {data['metadata']['temperature_min']:.1f} - "
                        f"{data['metadata']['temperature_max']:.1f} K")
        self.logger.info(f"  Analysis type: {data['metadata']['analysis_type']}")

        return data

    def _get_output_path(self, filename: str) -> Path:
        raise NotImplementedError("RstDataSource only supports reading")

    def _write_impl_temp_file(self, data: Dict[str, Any], output_path: Path) -> None:
        raise NotImplementedError("RstDataSource only supports reading")

    def should_skip(self, filename: str) -> bool:
        return False

# Step 4: Implement the transformation

To write to Zarr, we have to convert the in-memory data to a format that's ready to be written to be Zarr.

You can do that by following the code below.

In [14]:
from typing import Any, Dict

import numpy as np
import zarr

from physicsnemo_curator.etl.data_transformations import DataTransformation
from physicsnemo_curator.etl.processing_config import ProcessingConfig


class RstToZarrTransformation(DataTransformation):
    """
    Transform thermal simulation RST data into Zarr-optimized format.
    
    This transformation takes thermal analysis data (coordinates, temperature, heat_flux)
    and prepares it for efficient storage in Zarr format with:
    - Appropriate chunking for parallel access
    - Compression optimized for floating-point scientific data
    - Type conversion to float32 for storage efficiency
    """

    def __init__(self, cfg: ProcessingConfig, chunk_size: int = 1000, compression_level: int = 3):
        """
        Initialize the transformation.
        
        Args:
            cfg: Processing configuration
            chunk_size: Number of points per chunk (affects I/O performance)
        """
        super().__init__(cfg)
        self.chunk_size = chunk_size
        self.compressor = zarr.codecs.BloscCodec(
            cname="zstd",
            clevel=compression_level,
            shuffle=zarr.codecs.BloscShuffle.shuffle,
        )

    def transform(self, data: Dict[str, Any]) -> Dict[str, Any]:
        """
        Transform thermal RST data into Zarr-ready format.
        
        Args:
            data: Dictionary containing:
                - coordinates: (N, 3) array of node positions
                - temperature: (N,) array of temperature values
                - heat_flux: (N, 3) array of heat flux vectors
                - metadata: simulation metadata dict
                - filename: source filename
        
        Returns:
            zarr_data: Dictionary with arrays prepared for Zarr writing
        """
        self.logger.info(f"Transforming thermal data from {data['filename']}...")

        # Extract numpy arrays from the data source
        coords = data['coordinates']
        temperature = data['temperature']
        heat_flux = data['heat_flux']
        
        num_points = len(coords)
        
        # Determine chunk size (do not exceed total points)
        chunk_points = min(self.chunk_size, num_points)
        
        self.logger.info(f"  Processing {num_points} nodes with chunk size {chunk_points}")

        # Prepare Zarr-ready data structure
        zarr_data = {
            'coordinates': {},
            'temperature': {},
            'heat_flux': {},
            'metadata': data['metadata'].copy()  # Copy to avoid modifying original
        }

        # 1. Transform Coordinates (N, 3)
        zarr_data['coordinates'] = {
            'data': coords.astype(np.float32),
            'chunks': (chunk_points, 3),
            'compressors': (self.compressor,),
            'dtype': np.float32
        }

        # 2. Transform Temperature (N,) - scalar field
        zarr_data['temperature'] = {
            'data': temperature.astype(np.float32),
            'chunks': (chunk_points,),
            'compressors': (self.compressor,),
            'dtype': np.float32
        }

        # 3. Transform Heat Flux (N, 3) - vector field
        zarr_data['heat_flux'] = {
            'data': heat_flux.astype(np.float32),
            'chunks': (chunk_points, 3),
            'compressors': (self.compressor,),
            'dtype': np.float32
        }
        
        # Add curation metadata
        zarr_data['metadata']['curator_chunk_size'] = chunk_points
        zarr_data['metadata']['compression_type'] = 'zstd'
        zarr_data['metadata']['compression_level'] = self.compressor.clevel
        
        self.logger.info(f"  Transformation complete for {data['filename']}")
        
        return zarr_data

# Step 5: Implement the Zarr sink

The Zarr sink takes in the transformed data, and writes it to disk in an efficient manner.
This can be achieved by the script below.

In [ ]:
from pathlib import Path
from typing import Any, Dict, List

import zarr  # noqa: F811
from zarr.storage import LocalStore

from physicsnemo_curator.etl.data_sources import DataSource
from physicsnemo_curator.etl.processing_config import ProcessingConfig


class ZarrDataSource(DataSource):
    """DataSource for writing to Zarr stores."""

    def __init__(self, cfg: ProcessingConfig, output_dir: str):
        """Initialize the Zarr data source.

        Args:
            cfg: Processing configuration
            output_dir: Directory to write Zarr stores
        """
        super().__init__(cfg)
        self.output_dir = Path(output_dir)
        # Create output directory if it doesn't exist
        self.output_dir.mkdir(parents=True, exist_ok=True)

    def get_file_list(self) -> List[str]:
        """Not implemented - this DataSource only writes."""
        raise NotImplementedError("ZarrDataSource only supports writing")

    def read_file(self, filename: str) -> Dict[str, Any]:
        """Not implemented - this DataSource only writes."""
        raise NotImplementedError("ZarrDataSource only supports writing")

    def _get_output_path(self, filename: str) -> Path:
        """Get the output path for a given filename.

        Args:
            filename: Name of the file to process

        Returns:
            Path object representing the output location.
        """
        return self.output_dir / f"{filename}.zarr"

    def _write_impl_temp_file(self, data: Dict[str, Any], output_path: Path) -> None:
        """
        Implement actual data writing logic to a temporary Zarr store.

        This method is called by the write() method to write the data to a temporary Zarr store.
        The data is written to a temporary Zarr store and then renamed to the final output path.
        This is to improve the robustness of the write operation.

        Args:
            data: Transformed data from RstToZarrTransformation
            output_path: Path where data should be written (may be temporary)
        """
        # Create Zarr store
        self.logger.info(f"Creating Zarr store: {output_path}")
        store = LocalStore(output_path)
        root = zarr.open_group(store=store, mode="w")

        # Store metadata as root attributes
        if "metadata" in data:
            for key, value in data["metadata"].items():
                # Convert numpy types to Python types for JSON serialization
                if hasattr(value, "item"):  # numpy scalar
                    value = value.item()
                root.attrs[key] = value
            data.pop("metadata")

        # Write all arrays from the transformation
        for array_name, array_info in data.items():
            root.create_array(
                name=array_name,
                data=array_info["data"],
                chunks=array_info["chunks"],
                compressors=array_info["compressors"],  # Already a tuple from transformation
            )

        # Add some store-level metadata
        root.attrs["zarr_format"] = 3
        root.attrs["created_by"] = "physicsnemo-curator-tutorial"

        self.logger.info("Successfully created Zarr store")

    def should_skip(self, filename: str) -> bool:
        """Check if we should skip writing this store.

        Args:
            filename: Base filename to check

        Returns:
            True if store should be skipped (already exists)
        """
        store_path = self.output_dir / f"{filename}.zarr"
        exists = store_path.exists()

        if exists:
            self.logger.info(f"Skipping {filename} - Zarr store already exists")
            return True

        return False


# Step 6: Create a script to run the pipeline
PhysicsNeMo-Curator has a central orchestrator, to help you orchestrate your ETL pipeline. However, we need to create a script to instantiate the various components defined above, pass it to the orchestrator (along with multiprocessing context) and run the pipeline. The following code snippet can also be found in ./run_etl.py.

In [ ]:
import hydra
from hydra.utils import instantiate
from omegaconf import DictConfig

# PhysicsNeMo Curator Imports
from physicsnemo_curator.etl.etl_orchestrator import ETLOrchestrator
from physicsnemo_curator.etl.processing_config import ProcessingConfig
from physicsnemo_curator.utils import utils as curator_utils


@hydra.main(version_base="1.3", config_path="./config", config_name="st_pydpf_config")
def main(cfg: DictConfig) -> None:
    """
    Main ETL pipeline execution.
    
    Args:
        cfg: Hydra configuration object loaded from YAML
    """
    print("=" * 70)
    print("Ansys Thermal RST to Zarr ETL Pipeline")
    print("=" * 70)
    
    # 1. Setup Multiprocessing
    curator_utils.setup_multiprocessing()
    
    # 2. Parse Processing Config
    processing_config = ProcessingConfig(**cfg.etl.processing)
    print(f"Processing with {processing_config.num_processes} processes")

    # 3. Instantiate ETL Components
    print("\nInitializing ETL components...")
    
    # Source: Reads thermal RST files
    source = instantiate(cfg.etl.source, processing_config)
    print(f"  Source: {cfg.etl.source._target_}")
    print(f"  Input directory: {cfg.etl.source.input_dir}")
    
    # Sink: Writes Zarr stores
    sink = instantiate(cfg.etl.sink, processing_config)
    print(f"  Sink: {cfg.etl.sink._target_}")
    print(f"  Output directory: {cfg.etl.sink.output_dir}")

    # Transformations: RST to Zarr conversion
    cfgs = {k: {"_args_": [processing_config]} for k in cfg.etl.transformations.keys()}
    transformations = instantiate(cfg.etl.transformations, **cfgs)
    print(f"  Transformations: {list(cfg.etl.transformations.keys())}")

    # 4. Run ETL Orchestrator
    print("\n" + "=" * 70)
    print("Starting ETL Pipeline...")
    print("=" * 70 + "\n")
    
    orchestrator = ETLOrchestrator(
        source=source,
        sink=sink,
        transformations=transformations,
        processing_config=processing_config,
    )
    orchestrator.run()
    
    print("\n" + "=" * 70)
    print("ETL Pipeline Complete!")
    print("=" * 70)


if __name__ == "__main__":
    main()


# Step 7: Create a config

Now we'll tie everything together with a configuration file and run the complete pipeline. This config is present in [./config/rst_to_zarr.yaml](./config/rst_to_zarr.yaml).

In [ ]:
etl:
  processing:
    num_processes: 4  # Adjust based on available CPU cores
    args: {}

  # Source: Read thermal simulation .rst files
  source:
    _target_: rst_data_source.RstDataSource
    _convert_: all
    input_dir: ???  # Specify via CLI: etl.source.input_dir=path/to/rst/files

  # Transformations: Convert RST thermal data to Zarr-ready format
  transformations:
    rst_to_zarr:
      _target_: rst_to_zarr_transformation.RstToZarrTransformation
      _convert_: all
      chunk_size: 1000          # Points per chunk (tune for your data size)
      compression_level: 5      # Zstd level (1-9, higher = more compression)

  # Sink: Write Zarr stores to disk
  sink:
    _target_: zarr_data_source.ZarrDataSource
    _convert_: all
    output_dir: ???  # Specify via CLI: etl.sink.output_dir=path/to/output

# Step 8: Run the ETL Pipeline

Now you can run the complete pipeline using the run_etl.py file we just created above.

In [17]:
!python run_etl.py --config-dir ./ \
  --config-name rst_to_zarr \
  etl.source.input_dir=mock_thermal_data \
  etl.sink.output_dir=output_zarr

Ansys Thermal RST to Zarr ETL Pipeline
Processing with 4 processes

Initializing ETL components...
  Source: rst_data_source.RstDataSource
  Input directory: mock_thermal_data
  Sink: zarr_data_source.ZarrDataSource
  Output directory: output_zarr
  Transformations: ['rst_to_zarr']

Starting ETL Pipeline...

[2025-12-11 21:43:08,360][physicsnemo_curator.utils.utils][INFO] - Starting ETL pipeline
[2025-12-11 21:43:08,364][RstDataSource][INFO] - Found 5 .rst files to process
Processing files: 100%|█████████████████████████| 5/5 [00:00<00:00,  5.50file/s]
[2025-12-11 21:43:09,274][physicsnemo_curator.utils.utils][INFO] - 
Processing Summary:
[2025-12-11 21:43:09,274][physicsnemo_curator.utils.utils][INFO] - Number of processes: 4
[2025-12-11 21:43:09,274][physicsnemo_curator.utils.utils][INFO] - Total wall clock time: 0.91 seconds

ETL Pipeline Complete!


**What Happens During Execution:**

1. **Processing Phase**: For each file:
   - RstDataSource reads the rst file
   - RstToZarrTransformation converts it to Zarr-optimized format
   - ZarrDataSource writes the result to a `.zarr` store

3. **Parallel Execution**: Uses 4 processes to handle multiple files simultaneously

4. **Output**: Creates individual Zarr stores for each input file

**Expected Output Structure:**

After running, you'll have:

```bash
output_zarr/
├── run_001.zarr/
│   ├── coordinates/
│   ├── temperature/
│   ├── velocity/
│   ├── velocity_magnitude/
│   └── .zattrs (metadata)
├── run_002.zarr/
├── run_003.zarr/
├── run_004.zarr/
└── run_005.zarr/
```

**Verify the Results:**

You can inspect the output using this snippet:

In [ ]:
import zarr  # noqa: F811

# Open one of the Zarr stores
store = zarr.open("./output_zarr/thermal_sim_001.zarr", mode="r")

print("=" * 60)
print("Zarr Store Contents: thermal_sim_001.zarr")
print("=" * 60)

# List all arrays
print("\n📊 Arrays in store:")
for key in store.keys():
    print(f"  - {key}")

# Show array shapes and dtypes
print("\n📏 Array Details:")
for key in store.keys():
    array = store[key]
    print(f"  {key}:")
    print(f"    Shape: {array.shape}")
    print(f"    Dtype: {array.dtype}")
    print(f"    Chunks: {array.chunks}")

# Show metadata
print("\n📋 Metadata:")
for key, value in store.attrs.items():
    if isinstance(value, float):
        print(f"  {key}: {value:.2f}")
    else:
        print(f"  {key}: {value}")

# Show sample data from temperature
print("\n🌡️  Temperature Sample (first 5 values):")
print(f"  {store['temperature'][:5]}")

print("\n💨 Heat Flux Sample (first 3 vectors):")
print(f"  {store['heat_flux'][:3]}")

Zarr Store Contents: thermal_sim_001.zarr

📊 Arrays in store:
  - temperature
  - heat_flux
  - coordinates

📏 Array Details:
  heat_flux:
    Shape: (1728, 3)
    Dtype: float32
    Chunks: (1000, 3)
  temperature:
    Shape: (1728,)
    Dtype: float32
    Chunks: (1000,)
  coordinates:
    Shape: (1728, 3)
    Dtype: float32
    Chunks: (1000, 3)

📋 Metadata:
  simulation_name: thermal_sim_001
  num_nodes: 1728
  num_elements: 1331
  mesh_resolution: [12, 12, 12]
  scenario: heat_source
  units: SI
  temperature_units: Kelvin
  heat_flux_units: W/m^2
  coordinate_units: meters
  time_step: 1
  analysis_type: Steady-State Thermal
  solver: Mock Thermal Solver v1.0
  temperature_min: 294.86
  temperature_max: 395.47
  temperature_mean: 312.98
  curator_chunk_size: 1000
  compression_type: zstd
  compression_level: 5
  zarr_format: 3
  created_by: physicsnemo-curator-tutorial

🌡️  Temperature Sample (first 5 values):
  [301.25833 302.9406  303.90695 300.95227 302.8256 ]

💨 Heat Flux Sam